### Install yelpapi

In [1]:
# Install yelpapi
#!pip install yelpapi

### Install TQDM

In [2]:
# Install tqdm_notebook
#!pip install tqdm

In [3]:
# Importing .json file in .secret folder & others
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports

import os, json, math, time

# Importing YelpAPI
from yelpapi import YelpAPI

# Importing tqdm notebook
from tqdm.notebook import tqdm_notebook

In [4]:
# Open .json file
with open('/Users/njfor/.secret/yelp_api.json') as f:
    login = json.load(f)
login.keys()

dict_keys(['client-id', 'api-key'])

In [5]:
# Create & Run yelp_api
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)
yelp_api

In [6]:
# Quick Test Query
results = yelp_api.search_query(location='Boston, MA',
                               term='Lobster Roll')
print(type(results))
results.keys()

<class 'dict'>


dict_keys(['businesses', 'total', 'region'])

### Define Search

In [7]:
# set API call parameters
LOCATION = 'Boston,MA'
TERM = 'Lobster Roll'

In [8]:
# Specifying JSON_FILE filename (can include a folder)
# include the search terms in the filename
JSON_FILE = "Data/results_in_progress_BOS_lobroll.json"
JSON_FILE

'Data/results_in_progress_BOS_lobroll.json'

## Check if file already exists

In [9]:
# Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist:
if file_exists ==False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name ONLYl!
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
    # Create the folder if the length is > 0
        os.makedirs(folder, exist_ok=True)
    
    ## INFORM USER & SAVE EMPTY LIST
    print(f'[i] {JSON_FILE} not found. Saving empty list to file.')
    
    # save an empty list
    with open(JSON_FILE, 'w') as f:
        json.dump([],f)
    
# If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/results_in_progress_BOS_lobroll.json not found. Saving empty list to file.


## Determine how many results are already in the file

In [10]:
## Load previous results & use len (length) of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'` {n_results} previous results found.')

` 0 previous results found.


In [11]:
# Use our yelp_api variable's search query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                               term=TERM,
                               offset=n_results)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [12]:
## How many results total?
total_results = results['total']
total_results

1400

In [13]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

In [14]:
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

70

## Add results_per_page to .json file

In [15]:
previous_results.extend(results['businesses'])
with open(JSON_FILE, 'w') as f:
    json.dump(previous_results, f)

### Set up a progress bar in our for loop

In [16]:
# for loop to create a progress bar
for i in tqdm_notebook(range(n_pages)):
    # adds 200 millisecond pause
    time.sleep(.2)

  0%|          | 0/70 [00:00<?, ?it/s]

### Create a for loop to call each page

In [19]:
## delete previous_results file & confirm it no longer exists
os.remove(JSON_FILE)
os.path.isfile(JSON_FILE)

False

In [20]:
## The file is deleted & no longer exists.

In [21]:
def create_json_file(JSON_FILE,  delete_if_exists=False):
    
    ## Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)
    
    ## If it DOES exist:
    if file_exists == True:
        
        ## Check if user wants to delete if exists
        if delete_if_exists==True:
            
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            ## delete file and confirm it no longer exits.
            os.remove(JSON_FILE)
            ## Recursive call to function after old file deleted
            create_json_file(JSON_FILE,delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")            
            
            
    ## If it does NOT exist:
    else:
        
        ## INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        
        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)
        
        ## If JSON_FILE included a folder:
        if len(folder)>0:
            # create the folder
            os.makedirs(folder,exist_ok=True)
        ## Save empty list to start the json file
        with open(JSON_FILE,'w') as f:
            json.dump([],f)  

In [22]:
## Create new JSON FILE
## Create a new empty json file (exist the previous if it exists)
create_json_file(JSON_FILE, delete_if_exists=True)
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
## How many results total?
total_results = results['total']
## How many did we get the details for?
results_per_page = len(results['businesses'])
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

[i] Data/results_in_progress_BOS_lobroll.json not found. Saving empty list to new file.
- 0 previous results found.


70

## for loop that stops if exceeds 1000 api calls

In [23]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)

  0%|          | 0/70 [00:00<?, ?it/s]

Exceeded 1000 api calls. Stopping loop.


# Convert .json to pandas dataframe

In [24]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,HEaQQAH6qyFXamKkpil91A,paulis-boston,Pauli's,https://s3-media2.fl.yelpcdn.com/bphoto/9hiaqb...,False,https://www.yelp.com/biz/paulis-boston?adjust_...,1405,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",4.5,"{'latitude': 42.363244212396, 'longitude': -71...","[pickup, delivery]",$$,"{'address1': '65 Salem St', 'address2': None, ...",+18572847064,(857) 284-7064,5623.058982
1,i6uHrxuS1D_6V8WhaWLy5A,lukes-lobster-back-bay-boston,Luke's Lobster Back Bay,https://s3-media1.fl.yelpcdn.com/bphoto/EnMeIE...,False,https://www.yelp.com/biz/lukes-lobster-back-ba...,1988,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",4.0,"{'latitude': 42.34869123638961, 'longitude': -...","[pickup, delivery]",$$,"{'address1': '75 Exeter Street', 'address2': '...",+18573504626,(857) 350-4626,4407.684543
2,xkLwAfgIYXYHzJF8C_fhCA,tonys-clam-shop-quincy,Tony's Clam Shop,https://s3-media2.fl.yelpcdn.com/bphoto/fuctCw...,False,https://www.yelp.com/biz/tonys-clam-shop-quinc...,685,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",4.0,"{'latitude': 42.27524349415289, 'longitude': -...",[],$$,"{'address1': '861 Quincy Shore Dr', 'address2'...",+16177735090,(617) 773-5090,5899.108532
3,y2w6rFaO0XEiG5mFfOsiFA,neptune-oyster-boston,Neptune Oyster,https://s3-media4.fl.yelpcdn.com/bphoto/qzU7C1...,False,https://www.yelp.com/biz/neptune-oyster-boston...,5924,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",4.5,"{'latitude': 42.36321, 'longitude': -71.05594}",[delivery],$$$,"{'address1': '63 Salem St', 'address2': '', 'a...",+16177423474,(617) 742-3474,5622.373454
4,h4Un3Tw0yKn7uCLseO7Bvg,yankee-lobster-boston,Yankee Lobster,https://s3-media2.fl.yelpcdn.com/bphoto/f7Y0dH...,False,https://www.yelp.com/biz/yankee-lobster-boston...,2040,"[{'alias': 'seafoodmarkets', 'title': 'Seafood...",4.0,"{'latitude': 42.347825689583885, 'longitude': ...","[pickup, delivery]",$$,"{'address1': '300 Northern Ave', 'address2': '...",+16173459799,(617) 345-9799,4268.375988


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
995,4Th4Au8kRlL_uEXR-PeayA,sweet-rice-jp-boston,Sweet Rice JP,https://s3-media1.fl.yelpcdn.com/bphoto/RhonMr...,False,https://www.yelp.com/biz/sweet-rice-jp-boston?...,131,"[{'alias': 'thai', 'title': 'Thai'}, {'alias':...",4.0,"{'latitude': 42.31182, 'longitude': -71.11454}","[delivery, pickup]",$$,"{'address1': '697 Centre St', 'address2': None...",+16179422324,(617) 942-2324,4743.570827
996,mTyK9-_6CYY6XYAG56c68Q,carrie-nation-cocktail-club-boston-2,Carrie Nation Cocktail Club,https://s3-media2.fl.yelpcdn.com/bphoto/0b9ilW...,False,https://www.yelp.com/biz/carrie-nation-cocktai...,370,"[{'alias': 'tradamerican', 'title': 'American ...",3.5,"{'latitude': 42.35846, 'longitude': -71.061772}","[delivery, pickup]",$$,"{'address1': '11 Beacon St', 'address2': '', '...",+16172273100,(617) 227-3100,5106.974234
997,uYqizPy4eUSpwOha0RkVPg,no-9-park-boston,No. 9 Park,https://s3-media3.fl.yelpcdn.com/bphoto/q0HGQ7...,False,https://www.yelp.com/biz/no-9-park-boston?adju...,734,"[{'alias': 'french', 'title': 'French'}, {'ali...",4.0,"{'latitude': 42.357619051741864, 'longitude': ...",[delivery],$$$$,"{'address1': '9 Park St', 'address2': '', 'add...",+16177429991,(617) 742-9991,5021.289426
998,3UHLX26d_iPERGVzxfVLXQ,four-spoons-thai-inspired-cuisine-and-bar-newton,Four Spoons Thai Inspired Cuisine & Bar,https://s3-media1.fl.yelpcdn.com/bphoto/B7bSB5...,False,https://www.yelp.com/biz/four-spoons-thai-insp...,185,"[{'alias': 'thai', 'title': 'Thai'}, {'alias':...",4.0,"{'latitude': 42.33008, 'longitude': -71.19348}","[delivery, pickup]",$$,"{'address1': '796 Beacon St', 'address2': '', ...",+16173325888,(617) 332-5888,11405.553510
999,ALlXdnPBDoJY3NM-5iawLg,stoneforge-grill-south-easton,Stoneforge Grill,https://s3-media4.fl.yelpcdn.com/bphoto/dfPwY4...,False,https://www.yelp.com/biz/stoneforge-grill-sout...,249,"[{'alias': 'tradamerican', 'title': 'American ...",3.0,"{'latitude': 42.051126, 'longitude': -71.085041}","[delivery, pickup]",$$,"{'address1': '10 Roosevelt Cir', 'address2': '...",+15082380550,(508) 238-0550,29175.978920


## Check & remove any dupes

In [26]:
# check for duplicate results
final_df.duplicated(subset='id').sum()

1

In [27]:
# drop the dupes & sanity-check
final_df = final_df.drop_duplicates(subset='id')
final_df.duplicated(subset='id').sum()

0

## Save final_df to a .csv file or .csv.gz

In [28]:
# save the final results to a compressed csv (.csv.gz)
final_df.to_csv('Data/final_results_BOS_lobroll.csv.gz', compression='gzip', index=False)